In [1]:
import numpy as np
import pandas as pd

In [60]:
df1=pd.read_csv("../r2Data/user_vendor_series_data.csv",usecols=['user_id','business_id','stars_x','text','city','latitude','longitude','review_count','categories','name'],nrows=100000,keep_default_na=False)

## Buliding basic collaborative model

In [15]:
df1.drop(columns=['address','useful','funny','cool'],inplace=True)

In [33]:
df2=df1.iloc[:20000,:]

In [143]:
df1.head(3)

,user_id,business_id,stars_x,text,name,city,latitude,longitude,review_count,categories
0,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,"If you decide to eat here, just be aware it is...",Turning Point of North Wales,North Wales,40.210196,-75.223639,169,"Restaurants, Breakfast & Brunch, Food, Juice B..."
1,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,I've taken a lot of spin classes over the year...,Body Cycle Spinning Studio,Philadelphia,39.952103,-75.172753,144,"Active Life, Cycling Classes, Trainers, Gyms, ..."
2,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,Family diner. Had the buffet. Eclectic assortm...,Kettle Restaurant,Tucson,32.207233,-110.980864,47,"Restaurants, Breakfast & Brunch"


In [62]:
pivot_table_df=pd.pivot_table(df1,index=['name'],columns=['user_id'],values='stars_x')

In [63]:
pivot_table_df.fillna(value=0,inplace=True)

In [64]:
pivot_table_df

user_id,---2PmXbF47D870stH1jqA,--4AjktZiHowEIBCMd4CZA,--E0uVPphTORm_OiZ5KCvA,--KMTwCrhKKUmr7riuS4WQ,--S8M395r8NtOCvS2LRfDw,--UizzbnQlZg7bEv2oXEyg,--WhhlIim6ISY5yhFYZreg,--_H9j6ggxvqhh9nPofZwg,--_r6E98SNIrGU7weyNxbw,--ccVMj2PN6Z9qtdOdlung,...,zzeRzizkihWHz9bVAvbcVw,zzgLgszu48af4NJULymzAA,zzgMuJgxmToqcJ5iu1TngQ,zziJLt25YU6dp01sewR-IQ,zzjThJ4A1m1N78gquSR_QA,zzljYL9c22HWRO3VvJ1_Pg,zzsPf8xNW11nd0B6MZqfRw,zzsqjDvanJhH9tn8NautOQ,zztkCqqgR6VntYbqio4UTQ,zzxUT-fFNXpp5i5xJHmvpA
name,,,,,,,,,,,,,,,,,,,,,
"""No Macarena"" DJ & Custom Music Service",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'feine,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
+1 Gaming,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1 Tippling Place,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1-275 Rest Area Manatee County Mile 7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
uBreakiFix,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
velofix,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
vybe urgent care,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
from scipy.sparse import csr_matrix

In [66]:
df_sparse=csr_matrix(pivot_table_df)

In [67]:
from sklearn.neighbors import NearestNeighbors
model1=NearestNeighbors(algorithm='brute')

In [68]:
model1.fit(df_sparse)

NearestNeighbors(algorithm='brute')

In [135]:
def recommend(ind):
    distanceS,recS=model1.kneighbors(pivot_table_df.iloc[ind,:].values.reshape(1,-1),n_neighbors=20)
    for i in range(len(recS[0])):
        print(pivot_table_df.index[recS[0][i]])

In [136]:
recommend(1000)

Boise International Market
All 4 Dance
ADT Security Services
3am Liquor
Associated Dental Care - Tucson N Campbell
A.B. Collier Wrecker Service
Arch Engraving
A1 RV Sales
Atlantic Vapor
Aqua-Tots Swim Schools - Eagle
American Family Care
Action Towing
Americas Best Value Inn Media Philadelphia
Alamo Rent A Car
Accurate Window & Screen
A Psychic and Tarot Card Readings
Amy & Barber Beauty Salon
Aloha Japanese Steakhouse
Affordable Lock & Security Solutions - Tampa
Amor Nail and Spa


In [138]:
recommend(0)

"No Macarena" DJ & Custom Music Service
All 4 Dance
ADT Security Services
A1 RV Sales
Arch Engraving
American Family Care
Affordable Lock & Security Solutions - Tampa
A Psychic and Tarot Card Readings
Associated Dental Care - Tucson N Campbell
Aqua-Tots Swim Schools - Eagle
Action Towing
Accurate Window & Screen
Americas Best Value Inn Media Philadelphia
Alamo Rent A Car
A.B. Collier Wrecker Service
3am Liquor
Atlantic Vapor
Aloha Japanese Steakhouse
Amor Nail and Spa
Amy & Barber Beauty Salon


In [142]:
df1['text']=df1['text']+df1['categories']

In [149]:
df1['categories'][886]

'Bars, Nightlife, Lounges, Cocktail Bars'

## Content based recommendation

In [5]:
vdf=pd.read_json("../r2Data/yelp/yelp_academic_dataset_review.json",nrows=200000,lines=True)

In [7]:
udf=pd.read_json('../r2Data/yelp/yelp_academic_dataset_business.json',lines=True)

In [160]:
vdf

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
...,...,...,...,...,...,...,...,...,...
99995,pAEbIxvr6ebx2bHc1XvguA,SMH5CeiLvKx61lKwtLZ_PA,lV0k3BnslFRkuWD_kbKd0Q,4,0,0,0,Came here for lunch with a group. They were bu...,2018-05-30 22:28:56
99996,xH1AoE-4nf2ECGQJRjO4_g,2clTdtp-BjphxLjN83CpUA,G0xz3kyRhRi6oZl7KfR0pA,1,1,0,0,The equipment is so old and so felty! I just u...,2015-04-05 23:31:52
99997,GatIbXTz-WDru5emONUSIg,MRrN6DH3QGCFcDv5RENYVg,C4lZdhasjZVQyDlOiXY1sA,4,0,0,0,This is one of my favorite Mexican restaurants...,2016-06-04 00:59:15
99998,6NfkodAdhvI89xONXuBC3A,rnNQzeKJbvqVCsYsL10mkQ,dChRGpit9fM_kZK5pafNyA,2,0,0,0,Came here for brunch - had an omlette ($19 + t...,2018-06-11 12:45:08


In [8]:
udf

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,AB,T6J 5H2,53.468419,-113.492054,3.0,13,1,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","Nail Salons, Beauty & Spas","{'Monday': '10:0-19:30', 'Tuesday': '10:0-19:3..."
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Pets, Nurseries & Gardening, Pet Stores, Hobby...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3..."
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Shopping, Jewelry, Piercing, Toy Stores, Beaut...",None
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,1,"{'BusinessParking': '{'garage': False, 'street...","Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ..."
